In [1]:
# Importar librerias 
import matplotlib.pyplot as plt

import numpy as np
import os
import cv2

from tensorflow.keras.models import Sequential
# Tipos de capas que pueden tener las redes 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Definir directorios de los datos
# Entrenamiento
train_directory = 'CarneDataset/train'
# Test
test_directory = 'CarneDataset/test'
# Parametros de ejecucion
test_size=0.2
random_state=43
# Definiciòn de epocas
epochs=10
# Definiciòn de numero de imagenes que se le pasa a la red
batch_size=32

# Contar el  numero de archivos de un directorio
def fcount(path):
    count = 0
    for f in os.listdir(path):
        if os.path.isfile(os.path.join(path, f)):
            count += 1
            
    return count

# Guardar las imagenes de entrenamiento y los nombres de los archivos en listas de entrenamiento y de etiquetas
#  Si el directorio esta vacio no se guarda la lista
x_train_images = []
y_train_labels = []
y_train_labels1 = []

# Leer las imágenes de test y almacenar en las listas de imagenes y etiquetas

for label in os.listdir(train_directory):
    label_directory = os.path.join(train_directory, label)
    if (fcount(label_directory)) > 0:
        for filename in os.listdir(label_directory):
            img_path = os.path.join(label_directory, filename)
            img = cv2.imread(img_path)
            x_train_images.append(img)
            y_train_labels.append(label)
            y_train_labels1.append(label)

# Leer las imágenes de test y almacenar en las listas de imagenes y etiquetas
x_test_images = []
y_test_labels = []
for label in os.listdir(test_directory):
    label_directory = os.path.join(test_directory, label)
    if (label in y_train_labels1):    
        for filename in os.listdir(label_directory):
            img_path = os.path.join(label_directory, filename)
            img = cv2.imread(img_path)
            x_test_images.append(img)
            y_test_labels.append(label)
  
      
# Convertir las listas de imágenes y etiquetas a matrices NumPy
x_train_images = np.array(x_train_images)
y_train_labels = np.array(y_train_labels)

# Convertir las listas de imágenes y etiquetas de prueba a matrices NumPy
x_test_images = np.array(x_test_images)
y_test_labels = np.array(y_test_labels)

# Normalizar los valores de píxeles de las imágenes de prueba entre 0 y 1
x_test_images = x_test_images.astype('float32') / 255.0


# Codificar las etiquetas como números one hot encoding
label_encoder = LabelEncoder()
y_train_labels_encoded = label_encoder.fit_transform(y_train_labels)


# Codificar las etiquetas como números one hot encoding
y_test_labels_encoded = label_encoder.transform(y_test_labels)


# Dividir el conjunto de entrenamiento en entrenamiento y validación
x_train_images, x_val_images, y_train_labels_encoded, y_val_labels_encoded = train_test_split(x_train_images, y_train_labels_encoded, test_size=test_size, random_state=random_state)

# Normalizar los valores de píxeles entre 0 y 1
x_train_images = x_train_images.astype('float32') / 255.0
x_val_images = x_val_images.astype('float32') / 255.0


# Definir la arquitectura de la CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=x_train_images.shape[1:]))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# Clasificador
model.add(Flatten())
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo con los datos de entrenamiento y de validaciòn 
model.fit(x_train_images, y_train_labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=(x_val_images, y_val_labels_encoded))


# Realizar las predicciones en el conjunto de prueba
predictions = model.predict(x_test_images)
predicted_labels = np.argmax(predictions, axis=1)

# Calcular la matriz de confusión del modelo
confusion = confusion_matrix(y_test_labels_encoded, predicted_labels)
print('Matriz de confusión del modelo:')
print(confusion)

# Calcular la matriz de confusión del error en entrenamiento
train_predictions = model.predict(x_train_images)
train_predicted_labels = np.argmax(train_predictions, axis=1)
train_confusion = confusion_matrix(y_train_labels_encoded, train_predicted_labels)
print('Matriz de confusión del error en entrenamiento:')
print(train_confusion)

# Calcular la matriz de confusión del error en prueba
test_predictions = model.predict(x_test_images)
test_predicted_labels = np.argmax(test_predictions, axis=1)
test_confusion = confusion_matrix(y_test_labels_encoded, test_predicted_labels)
print('Matriz de confusión del error en prueba:')
print(test_confusion)

Epoch 1/10
41/41 [==============================] - 211s 5s/step - loss: 1.8678 - accuracy: 0.5329 - val_loss: 1.2494 - val_accuracy: 0.6208
Epoch 2/10
 4/41 [=>............................] - ETA: 2:55 - loss: 1.0642 - accuracy: 0.6875

KeyboardInterrupt: 